В этом блокноте мы создаем синтетические данные для обучения GLiNER. 
План:
1) Мы берем случайную статью из википедии. 
2) Просим Большую Языковую Модель выписать все сущности из текста в определнном формате
3) Парсим выход

In [6]:
import json
# У мистраля бесплатное API для LLM, работает без VPN, но можно переписать на другое при желании
from mistralai import Mistral
import traceback # в случае ошибки, при использовании try-except, выводит полный лог
import time

# Сохранять предсказания из dict, в json
def save_data_to_file(data, filepath):
    """Saves the processed data to a JSON file."""
    with open(filepath, 'w') as f:
        json.dump(data, f)

# нужно вставить свой ключ
api_key = "mXhjdibfW9ay0Crp7LQ6ZHGlP9cTVFnN"  # дарю

# выбрать модель 
# model = "mistral-large-latest"
model = "open-mistral-nemo"

client = Mistral(api_key=api_key)

def generate(prompt):

    chat_response = client.chat.complete(
        model= model,
        messages = [
            {
                "role": "user",
                "content": prompt,
            },
        ]
    )
    return chat_response.choices[0].message.content
    # outputs = llm.generate([create_json_prompt_for_synthetic_data(**kwargs)], sampling_params)
    # return json.loads(outputs[0].outputs[0].text)
    

In [7]:
# Инструкция, которая пойдет в модель. Модель просят извлечь из данного текста все сущности, и написать свой ответ в определенном json формате.

def create_json_prompt_for_text_analysis(text):
    prompt = """
**Objective:**
Analyze the given text and extract named entities. Each entity should be meticulously labeled according to its type for straightforward extraction.

**Format Requirements:**
- The output should be formatted in JSON, containing the text and the corresponding entities list.
- Each entity in the text should be accurately marked and annotated in the 'entities' list.
- Meticulously follow all the listed attributes.

**Entity Annotation Details:**
- All entity types must be in lowercase. For example, use "type" not "TYPE".
- Entity types can be multiwords separate by space. For instance, use "entity type" rather than "entity_type".
- Entities spans can be nested within other entities.
- A single entity may be associated with multiple types. list them in the key "types".

**Output Schema:**

```json
{
  "text": "{text}",
  "entities": [
    {"entity": "entity name", "types": ["type 1", "type 2", ...]},
    ...
  ]
}
```

**Example:**

Input text was:
"Вакансия: Инженер по машинному обучению в компании ООО \"ТехноСофт\". Требуется опыт работы с Python и TensorFlow. Расположение: Москва, Россия. Зарплата: 150,000 рублей в месяц. Контактное лицо: Иван Иванов, email: ivan.ivanov@techsoft.ru."

Correct output:
```json
{
  "text": "Вакансия: Инженер по машинному обучению в компании ООО \"ТехноСофт\". Требуется опыт работы с Python и TensorFlow. Расположение: Москва, Россия. Зарплата: 150,000 рублей в месяц. Контактное лицо: Иван Иванов, email: ivan.ivanov@techsoft.ru.",
  "entities": [
    {"entity": "Инженер по машинному обучению", "types": ["Должность"]},
    {"entity": "ООО \"ТехноСофт\"", "types": ["Название компании"]},
    {"entity": "Python", "types": ["Язык программирования"]},
    {"entity": "TensorFlow", "types": ["Программное обеспечение"]},
    {"entity": "Москва, Россия", "types": ["Место"]},
    {"entity": "150,000 рублей в месяц", "types": ["Зарплата"]},
    {"entity": "Иван Иванов", "types": ["Имя"]},
    {"entity": "ivan.ivanov@techsoft.ru", "types": ["Email"]}
  ]
}
```

"""

    # Adding the dynamically created text to the prompt
    prompt += f"""
Text: 
"{text}">
"""

    return prompt

# Пример использования функции
text = "Вакансия: Инженер по машинному обучению в компании ООО \"ТехноСофт\". Требуется опыт работы с Python и TensorFlow. Расположение: Москва, Россия. Зарплата: 150,000 рублей в месяц. Контактное лицо: Иван Иванов, email: ivan.ivanov@techsoft.ru."
# print(create_json_prompt_for_text_analysis(text))


## Берем случайное краткое содержание из википедии

In [8]:
import requests
import random

# Получаем случайную статью из Википедии
response = requests.get('https://en.wikipedia.org/api/rest_v1/page/random/summary')
data = response.json()

# Получаем текст статьи
text = data['extract']

# Ограничиваем текст до нескольких предложений
sentences = text.split('. ')
if len(sentences) > 3:
    text = '. '.join(sentences[:3]) + '.'

# Выводим текст
print(text)


Blue Water High is an Australian television drama series, broadcast by the Australian Broadcasting Corporation on ABC1 and on Austar/Foxtel Nickelodeon channel in Australia and on various channels in many other countries. Each season follows the lives of a young group of students at Solar Blue, a high-performance surf academy where several lucky 16-year-olds are selected for a 12-month-long surfing program at Bilgola Beach, Sydney.


## Функции, которые распарсят ответ модели, и переведут его в формат, который подходит для обучения модели. 

Это переделанный код из примеров с гитхаба. Тут не очень красиво. 

Эти функции переформатируют ответы LLM, в формат, который поддерживает GLiNER для обучения. 

In [9]:
import re

# Эта функция делит на слова
def tokenize_text(text):
    """Tokenize the input text into a list of tokens."""
    return re.findall(r'\w+(?:[-_]\w+)*|\S', text)

# Находит срез (номера слов начала и конца), для слов которые мы нашли
# По сути тут просто алгоритм поиска, но написанный на коленке, для работы с массивами
def extract_entities(data):
    all_examples = []

    for dt in data:

        # Attempt to extract entities; skip current record on failure
        try:
            tokens = tokenize_text(dt['text'])
            ents = [(k["entity"], k["types"]) for k in dt['entities']]
        except:
            continue

        spans = []
        for entity in ents:
            entity_tokens = tokenize_text(str(entity[0]))

            # Find the start and end indices of each entity in the tokenized text
            for i in range(len(tokens) - len(entity_tokens) + 1):
                if " ".join(tokens[i:i + len(entity_tokens)]).lower() == " ".join(entity_tokens).lower():
                    for el in entity[1]:
                        spans.append((i, i + len(entity_tokens) - 1, el.lower().replace('_', ' ')))

        # Append the tokenized text and its corresponding named entity recognition data
        all_examples.append({"tokenized_text": tokens, "ner": spans})

    return all_examples

### Generate outputs

In [11]:
N_SAMPLES = 100

processed_output = []

output_file = "gliner.json"

with open(output_file, "r") as f:
    processed_output = json.load(f)

print('Dataset size:', len(processed_output))


for i in range(N_SAMPLES):
    # Получаем случайную статью из Википедии
    try:
        response = requests.get('https://ru.wikipedia.org/api/rest_v1/page/random/summary')
        data = response.json()

        # Получаем текст статьи
        text = data['extract']
    except:
        continue

    # Ограничиваем текст до нескольких предложений
    sentences = text.split('. ')
    if len(sentences) > 3:
        text = '. '.join(sentences[:3]) + '.'

    prompt = create_json_prompt_for_text_analysis(text)

    answer = generate(prompt)
    print(answer)

    answer = answer.replace('```json', '').replace('```', '')

    try:
        js = json.loads(answer.strip())
    except:
        print("Error while parsing json!!!")
        print("Here is that faulty json:")
        print(answer)
        break


    # Attempt to extract entities; skip current record on failure
    try:
        tokens = tokenize_text(js['text'])
        ents = [(k["entity"], k["types"]) for k in js['entities']]
    except:
        print("Failed at entities step!")
        traceback.print_exc()
        break

    spans = []
    for entity in ents:
        entity_tokens = tokenize_text(str(entity[0]))

        # Find the start and end indices of each entity in the tokenized text
        for i in range(len(tokens) - len(entity_tokens) + 1):
            if " ".join(tokens[i:i + len(entity_tokens)]).lower() == " ".join(entity_tokens).lower():
                for el in entity[1]:
                    spans.append((i, i + len(entity_tokens) - 1, el.lower().replace('_', ' ')))

        # Append the tokenized text and its corresponding named entity recognition data
    print(len(processed_output), '----', {"tokenized_text": tokens, "ner": spans})

    processed_output.append({"tokenized_text": tokens, "ner": spans})

    # break
    save_data_to_file(processed_output, output_file)

    time.sleep(2)
    

Dataset size: 0
```json
{
  "text": "Органические гидропероксиды — соединения состава ROOH, содержащие пероксидную группу O−O и являющиеся органическими производными пероксида водорода, в молекуле которого один из атомов водорода замещён на углеводородный радикал R.",
  "entities": [
    {"entity": "Органические гидропероксиды", "types": ["Химическое соединение"]},
    {"entity": "ROOH", "types": ["Химическая формула"]},
    {"entity": "пероксидная группа O−O", "types": ["Химическая группа"]},
    {"entity": "пероксида водорода", "types": ["Химическое соединение"]},
    {"entity": "углеводородный радикал R", "types": ["Химический радикал"]}
  ]
}
```
0 ---- {'tokenized_text': ['Органические', 'гидропероксиды', '—', 'соединения', 'состава', 'ROOH', ',', 'содержащие', 'пероксидную', 'группу', 'O', '−', 'O', 'и', 'являющиеся', 'органическими', 'производными', 'пероксида', 'водорода', ',', 'в', 'молекуле', 'которого', 'один', 'из', 'атомов', 'водорода', 'замещён', 'на', 'углеводородный', '